In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F


/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
3


In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou


def apply_fisheye_distortion(images, masks, label):
    # 이미지 크기 가져오기
    batch, channel, height, width = images.shape

    # 카메라 매트릭스 생성
    focal_length = width / 4
    center_x = width / 2
    center_y = height / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)

    # 왜곡 계수 생성
    dist_coeffs = np.array([0, 0.2 * label, 0, 0], dtype=np.float32)

    # 왜곡 보정
    undistorted_images = []
    undistorted_masks = []

    for i in range(batch):
        image = images[i].permute(1, 2, 0).cpu().numpy()  # 텐서를 NumPy 배열로 변환
        mask = masks[i].cpu().numpy()
        undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs)
        undistorted_mask = cv2.undistort(mask, camera_matrix, dist_coeffs)
        undistorted_mask = np.round(undistorted_mask).astype(np.uint8)
        undistorted_mask[undistorted_mask > 12] = 12

        # 다시 텐서로 변환
        undistorted_image = torch.from_numpy(undistorted_image).permute(2, 0, 1).float().to(device)
        undistorted_mask = torch.from_numpy(undistorted_mask).long().to(device)

        undistorted_images.append(undistorted_image)
        undistorted_masks.append(undistorted_mask)

    undistorted_images = torch.stack(undistorted_images, dim=0)
    undistorted_masks = torch.stack(undistorted_masks, dim=0)

    return undistorted_images, undistorted_masks

In [4]:
import numpy as np
import cv2
import torch



## 데이터셋

In [5]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        directory_path = './data/224'
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path)
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = np.round(mask).astype(np.uint8)
        mask[mask > 12] = 12 #배경을 픽셀값 12로 간주
        mask += 1
        mask[mask == 13] = 0

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

transform = A.Compose(
    [   
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

## 모델 선언

In [6]:
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(self, x):
        return x.view_as(x)
    @staticmethod
    def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
        return grad_output * (-1)

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 4) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        #return torch.sigmoid(x)
        return x

class domain_linear(nn.Module):
    def __init__(self):
        super(domain_linear, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 1) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        #return torch.sigmoid(x)
        return x
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        
        # 3x3 convolution
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU()
        
        # 3x3 convolution
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        
        # Skip connection
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Adding the skip connection
        out += self.skip(identity)
        out = self.relu2(out)
        
        return out
#인코더 블럭
class Conv2(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv2,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        
        return x , p
class Conv3(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv3,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,in_channels)
        self.identityblock4 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        p = self.maxpool(x)
        
        return x , p
class Conv4(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv4,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,in_channels)
        self.identityblock4 = IdentityBlock(in_channels,in_channels)
        self.identityblock5 = IdentityBlock(in_channels,in_channels)
        self.identityblock6 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        x = self.identityblock5(x)
        x = self.identityblock6(x)
        p = self.maxpool(x)
        
        return x , p
class Conv5(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv5,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        
        return x , p
#디코더 블럭
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1) # output_padding 추가
        self.convblock1 = IdentityBlock(channels*2, channels)

    def forward(self, x, skip):
        x = self.upsample(x)
        if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
            x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #print("x",x.shape,"skip: ",skip.shape)
        return x

#Unet구조 middle의 xm값의 움직임에 주의
class Resnet34_Unet(nn.Module):
    def __init__(self,n_classes):
        super(Resnet34_Unet,self).__init__()
        self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.fbn1 = nn.BatchNorm2d(64)
        self.frelu1 = nn.ReLU()
        self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.conv2 = Conv2(64,128)
        self.conv3 = Conv3(128,256)
        self.conv4 = Conv4(256,512)
        self.conv5 = Conv5(512,1024)
        
        self.middleconv = IdentityBlock(1024,2048)
        self.dropout = nn.Dropout2d(0.1) #
           
        self.decoder5 = DecoderBlock(1024)
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.transpose = nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=1) # output_padding 추가
        
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        self.domain_classifier = domain_classifier()
        self.domain_linear = domain_linear()
        
    def forward(self,x):
        x = self.fconv1(x)#3->64
        x0 = self.fbn1(x)
        x1 = self.frelu1(x)
        p = self.fmaxpooling(x1)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
        #print("conv1: ",x1.shape, "maxpooling: ",p.shape)
        x2,p = self.conv2(p)
        #print("conv2: ",x2.shape, "maxpooling: ",p.shape)
        x3,p = self.conv3(p)
        #print("conv3: ",x3.shape, "maxpooling: ",p.shape)
        x4,p = self.conv4(p)
        #print("conv4: ",x4.shape, "maxpooling: ",p.shape)
        x5,p = self.conv5(p)
        #print("conv5: ",x5.shape, "maxpooling: ",p.shape)
        
        xm = self.middleconv(p)#xm([8, 4096, 2, 2])
        #print("xm: ",xm.shape, "maxpooling: ",p.shape)
        xm = self.dropout(xm)
        
        x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
        x = self.decoder4(x,x4)#뉴런:1024*2->1024 
        x = self.decoder3(x,x3) #14
        x = self.decoder2(x,x2)#28
        x = self.decoder1(x,x1)#55
        x = self.transpose(x)
        
        #print(x.shape)
        #x = F.interpolate(x, size=(224, 224))
        x_c = self.segmap(x)
        x_d = self.domain_classifier(x)
        
        return x_c,x_d

In [7]:
# import pickle

# # 저장된 class_weights를 불러옵니다.
# class_weights_path = 'CLASS_WEIGHTS.pkl'

# with open(class_weights_path, 'rb') as file:
#     CLASS_WEIGHTS = pickle.load(file)

# print(CLASS_WEIGHTS)

## Loss Function

In [8]:
# loss function과 optimizer 정의

class DANN_Loss(nn.Module):
    def __init__(self):
        super(DANN_Loss, self).__init__()

        #self.CE = nn.CrossEntropyLoss(weight=CLASS_WEIGHTS) # 0~9 class 분류용
        self.CE = nn.CrossEntropyLoss()
        
        
    # result : DANN_CNN에서 반환된 값
    # label : 숫자 0 ~ 9에 대한 라벨
    # domain_num : 0(source) or 1(target)
    def forward(self, result, label, domain_num, alpha = 1):
        label_logits, domain_logits = result # DANN_CNN의 결과

        batch_size = domain_logits.shape[0]

        segment_loss = self.CE(label_logits, label) # class 분류 loss
        
        domain_target = torch.LongTensor([domain_num] * batch_size).to(device)

        domain_loss = self.CE(domain_logits, domain_target) # domain 분류 loss
        
        #domain_target = torch.FloatTensor([domain_num] * batch_size).unsqueeze(1).to(device)
        #domain_loss = self.BCE(domain_logits, domain_target) # domain 분류 loss
        
        loss = segment_loss + alpha * domain_loss
        # print("segment_mask : ", label.shape)
        # print("domain_answer : ", domain_target.shape)
        return (loss) ,segment_loss, domain_loss
    
class DANN_Loss_mse(nn.Module):
    def __init__(self):
        super(DANN_Loss_mse, self).__init__()
        self.CE = nn.CrossEntropyLoss()
        self.MSE = nn.MSELoss()

    def forward(self, result, label, domain_num, alpha = 1):
        label_logits, domain_logits = result # DANN_CNN의 결과

        batch_size = domain_logits.shape[0]

        segment_loss = self.CE(label_logits, label) # class 분류 loss
        
        domain_target = torch.LongTensor([domain_num] * batch_size).unsqueeze(1).to(device).float()      
        domain_loss = self.MSE(domain_logits, domain_target) # domain 분류 loss
        domain_loss = torch.sqrt(domain_loss)
        loss = segment_loss + alpha * domain_loss
        # print("segment_mask : ", label.shape)
        # print("domain_answer : ", domain_target.shape)
        return loss ,segment_loss, domain_loss
    

loss_fn = DANN_Loss().to(device)
#loss_fn = DANN_Loss_mse().to(device)

#criterion =nn.CrossEntropyLoss()
#domain_criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss(weight=class_weights)

In [9]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [10]:
LR = 0.001
EP = 35
BATCH_SIZE = 16
ACCMULATION_STEP = 1 
N_CLASSES = 13
alpha = 0.01
N_LABELS = 4
# model 초기화
model = Resnet34_Unet(n_classes = N_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

optimizer.zero_grad() 

source_dataset = CustomDataset(csv_file='./data/896_csv/train_source.csv', transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_target_dataset = CustomDataset(csv_file='./data/896_csv/val_source.csv', transform=transform)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [11]:
for epoch in range(EP):
    model.train()
    epoch_loss = 0
    seg_losses = 0
    domain_losses = 0
    alpha = 0.0045
    train_class_ious = [[],[],[],[]]
    train_ious = pd.DataFrame()
    print(alpha)
    for source_images, source_masks in tqdm(source_dataloader,desc=f"Epoch: {epoch+1}"):
        for label in range(N_LABELS): 
            source_image, source_mask = apply_fisheye_distortion(source_images, source_masks, label)
            source_image = source_image.float().to(device)
            source_mask = source_mask.long().to(device)
            source_outputs = model(source_image)
            
            optimizer.zero_grad()
            target_loss, seg_loss, domain_loss = loss_fn(source_outputs, source_mask, label, alpha = alpha)
            epoch_loss += target_loss.item()
            seg_losses +=  seg_loss.item()
            domain_losses += domain_loss.item()
            target_loss.backward()
            optimizer.step()
            # miou측정
            source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
            source_outputs = torch.argmax(source_outputs, dim=1).numpy()
            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
                train_class_ious[label].append(iou)
    
    for i in range(N_LABELS):
        buff = np.array(train_class_ious[i]).reshape(N_CLASSES, -1)
        buff =np.mean(buff, axis=1)
        print(f"\nLabel_{i}: IoU Scores Train") 
        for class_id, iou in enumerate(buff):
            print(f'Class{class_id:02d}: {iou:.4f}', end=" ")
            if (class_id+1) % 7 == 0:
                print()   
    print()    
    print(f"Train seg Loss: {(seg_losses/(N_LABELS*len(source_dataloader)))}")
    print(f"Train dom Loss: {(domain_losses/(N_LABELS*len(source_dataloader)))}")
    print(f"Train Loss: {(epoch_loss/(N_LABELS*len(source_dataloader)))}")
    print(f"Train mIoU: {np.mean(train_class_ious)}" )
    ################################################################
    # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    fish_val_class_ious = []
    val_epoch_loss = 0
    val_seg_loss = 0
    val_domain_loss = 0
    # valid
    with torch.no_grad():
        model.eval()
        for target_images, target_masks in tqdm(val_target_dataloader):
            label = 2.5
            target_images, target_masks = apply_fisheye_distortion(target_images, target_masks, label)
            target_images = target_images.float().to(device)
            target_masks = target_masks.long().to(device)

            target_outputs = model(target_images)

            target_loss, target_seg_loss, target_domain_loss = loss_fn(target_outputs, target_masks, 0, alpha = alpha)

            val_seg_loss +=  target_seg_loss.item()
            val_domain_loss += target_domain_loss.item()
            
            loss = target_loss

            val_epoch_loss += loss.item()

            # train 클래스별 IoU 계산
            target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
            target_outputs = torch.argmax(target_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
                fish_val_class_ious.append(iou)

    fish_val_class_ious = np.array(fish_val_class_ious).reshape(N_CLASSES, -1)
    fish_val_class_ious = np.mean(fish_val_class_ious, axis=1)
    print()
    print("--IoU Scores Fish val--")
    for class_id, iou in enumerate(fish_val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    fish_val_mIoU = np.mean(fish_val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Valid_Seg Loss: {(val_seg_loss/len(val_target_dataloader))}",f"Valid_dom Loss: {(val_domain_loss/len(val_target_dataloader))}")
    print(f"Valid Loss: {(val_epoch_loss/len(val_target_dataloader))}")
    print(f"Valid mIoU: {fish_val_mIoU}" )
    print("___________________________________________________________________________________________\n")

print("Hyperparamerters")
print(f"LR = {LR} | EP = {EP}, BATCH_SIZE = {BATCH_SIZE}, N_CLASSES = {N_CLASSES}, init_alpha = {alpha}, N_LABELS = {N_LABELS}")

0.0045


Epoch: 1: 100%|██████████| 138/138 [05:52<00:00,  2.56s/it]



Label_0: IoU Scores Train
Class00: 0.0194 Class01: 0.0346 Class02: 0.1190 Class03: 0.1662 Class04: 0.2415 Class05: 0.2226 Class06: 0.2596 
Class07: 0.2698 Class08: 0.2973 Class09: 0.3166 Class10: 0.3047 Class11: 0.3146 Class12: 0.3043 
Label_1: IoU Scores Train
Class00: 0.0233 Class01: 0.0302 Class02: 0.0597 Class03: 0.0752 Class04: 0.0917 Class05: 0.1020 Class06: 0.1113 
Class07: 0.1038 Class08: 0.1110 Class09: 0.1147 Class10: 0.1116 Class11: 0.1133 Class12: 0.1106 
Label_2: IoU Scores Train
Class00: 0.0223 Class01: 0.0285 Class02: 0.0527 Class03: 0.0662 Class04: 0.0776 Class05: 0.0863 Class06: 0.0927 
Class07: 0.0872 Class08: 0.0925 Class09: 0.0955 Class10: 0.0934 Class11: 0.0951 Class12: 0.0927 
Label_3: IoU Scores Train
Class00: 0.0209 Class01: 0.0274 Class02: 0.0490 Class03: 0.0614 Class04: 0.0704 Class05: 0.0778 Class06: 0.0829 
Class07: 0.0785 Class08: 0.0831 Class09: 0.0854 Class10: 0.0835 Class11: 0.0853 Class12: 0.0833 
Train seg Loss: 0.5946629162391891
Train dom Loss: 18.0

100%|██████████| 30/30 [00:35<00:00,  1.18s/it]



--IoU Scores Fish val--
Class0: 0.3342 Class1: 0.2581 Class2: 0.3115 Class3: 0.3414 Class4: 0.2861 Class5: 0.3174 Class6: 0.3498 
Class7: 0.2656 Class8: 0.3111 Class9: 0.3101 Class10: 0.2991 Class11: 0.2937 Class12: 0.3122 
Epoch1
Valid_Seg Loss: 0.2365197941660881 Valid_dom Loss: 22.81543566385905
Valid Loss: 0.3391892502705256
Valid mIoU: 0.30696148779697546
___________________________________________________________________________________________

0.0045


...


--IoU Scores Fish val--
Class0: 0.2884 Class1: 0.2261 Class2: 0.2715 Class3: 0.2919 Class4: 0.2408 Class5: 0.2741 Class6: 0.2983 
Class7: 0.2260 Class8: 0.2641 Class9: 0.2733 Class10: 0.2550 Class11: 0.2492 Class12: 0.2568 
Epoch34
Valid_Seg Loss: 0.31886899371941885 Valid_dom Loss: 21.918523089090982
Valid Loss: 0.41750234365463257
Valid mIoU: 0.2627259974396439
___________________________________________________________________________________________

0.0045


Epoch: 35: 100%|██████████| 138/138 [05:52<00:00,  2.55s/it]



Label_0: IoU Scores Train
Class00: 0.2640 Class01: 0.2643 Class02: 0.2718 Class03: 0.2748 Class04: 0.2723 Class05: 0.2682 Class06: 0.2762 
Class07: 0.2613 Class08: 0.2728 Class09: 0.2656 Class10: 0.2546 Class11: 0.2429 Class12: 0.2529 
Label_1: IoU Scores Train
Class00: 0.1007 Class01: 0.1008 Class02: 0.1016 Class03: 0.1032 Class04: 0.1010 Class05: 0.1020 Class06: 0.1053 
Class07: 0.1013 Class08: 0.1052 Class09: 0.1040 Class10: 0.0993 Class11: 0.0983 Class12: 0.0995 
Label_2: IoU Scores Train
Class00: 0.0849 Class01: 0.0851 Class02: 0.0856 Class03: 0.0868 Class04: 0.0852 Class05: 0.0855 Class06: 0.0885 
Class07: 0.0851 Class08: 0.0883 Class09: 0.0872 Class10: 0.0837 Class11: 0.0844 Class12: 0.0837 
Label_3: IoU Scores Train
Class00: 0.0762 Class01: 0.0764 Class02: 0.0767 Class03: 0.0782 Class04: 0.0766 Class05: 0.0764 Class06: 0.0795 
Class07: 0.0765 Class08: 0.0793 Class09: 0.0781 Class10: 0.0749 Class11: 0.0757 Class12: 0.0746 
Train seg Loss: 0.4696182020386492
Train dom Loss: 67.6

100%|██████████| 30/30 [00:37<00:00,  1.27s/it]


--IoU Scores Fish val--
Class0: 0.2661 Class1: 0.2081 Class2: 0.2589 Class3: 0.2713 Class4: 0.2223 Class5: 0.2618 Class6: 0.2699 
Class7: 0.2128 Class8: 0.2564 Class9: 0.2514 Class10: 0.2298 Class11: 0.2348 Class12: 0.2516 
Epoch35
Valid_Seg Loss: 0.37653429905573527 Valid_dom Loss: 18.430092112223306
Valid Loss: 0.45946970880031585
Valid mIoU: 0.24577917236310926
___________________________________________________________________________________________

Hyperparamerters
LR = 0.001 | EP = 35, BATCH_SIZE = 16, N_CLASSES = 13, init_alpha = 0.0045, N_LABELS = 4


In [12]:
torch.save(model.state_dict(), './data/resnet34_1107_1.pth')
